In [9]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

graph = Neo4jGraph()

In [4]:
from langchain_community.vectorstores import Neo4jVector

from langchain_openai import AzureOpenAIEmbeddings

embedding = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    azure_endpoint='https://lang-chain-dev.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15',
    azure_deployment='text-embedding-ada-002',
    openai_api_version='2023-05-15'
)

vector = Neo4jVector.from_existing_graph(
    embedding,
    node_label='__Entity__',
    text_node_properties=['id', 'description'],
    embedding_node_property='embedding'
)

In [ ]:
# ! pip3 install graphdatascience

In [5]:
from graphdatascience import GraphDataScience 

gds = GraphDataScience( 
    os.environ[ "NEO4J_URI" ], 
    auth=(os.environ[ "NEO4J_USERNAME" ], os.environ[ "NEO4J_PASSWORD" ]) 
)

/Users/andy/miniforge3/envs/lagch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
G, result = gds.graph.project(
    "entities",                   # Graph name
    "__Entity__",                 # Node projection
    "*",                          # Relationship projection
    nodeProperties=["embedding"]  # Configuration parameters
)

Loading: 100%|██████████| 100.0/100 [00:00<00:00, 393.63%/s]


In [7]:
similarity_threshold = 0.95

gds.knn.mutate(
  G,
  nodeProperties=['embedding'],
  mutateRelationshipType= 'SIMILAR',
  mutateProperty= 'score',
  similarityCutoff=similarity_threshold
)

ranIterations                                                             3
nodePairsConsidered                                                    7038
didConverge                                                            True
preProcessingMillis                                                       8
computeMillis                                                            47
mutateMillis                                                             47
postProcessingMillis                                                      0
nodesCompared                                                            29
relationshipsWritten                                                    152
similarityDistribution    {'min': 0.950439453125, 'p5': 0.95149612426757...
configuration             {'mutateProperty': 'score', 'jobId': '8dc6c77d...
Name: 0, dtype: object

In [8]:
gds.wcc.write(
    G,
    writeProperty="wcc",
    relationshipTypes=["SIMILAR"]
)

writeMillis                                                             72
nodePropertiesWritten                                                   29
componentCount                                                           6
componentDistribution    {'min': 1, 'p5': 1, 'max': 21, 'p999': 21, 'p9...
postProcessingMillis                                                    46
preProcessingMillis                                                      1
computeMillis                                                            9
configuration            {'writeProperty': 'wcc', 'jobId': 'adce5009-b3...
Name: 0, dtype: object

In [11]:
word_edit_distance = 3
potential_duplicate_candidates = graph.query(
    """MATCH (e:`__Entity__`)
    WHERE size(e.id) > 3 // longer than 3 characters
    WITH e.wcc AS community, collect(e) AS nodes, count(*) AS count
    WHERE count > 1
    UNWIND nodes AS node
    // Add text distance
    WITH distinct
      [n IN nodes WHERE apoc.text.distance(toLower(node.id), toLower(n.id)) < $distance 
                  OR node.id CONTAINS n.id | n.id] AS intermediate_results
    WHERE size(intermediate_results) > 1
    WITH collect(intermediate_results) AS results
    // combine groups together if they share elements
    UNWIND range(0, size(results)-1, 1) as index
    WITH results, index, results[index] as result
    WITH apoc.coll.sort(reduce(acc = result, index2 IN range(0, size(results)-1, 1) |
            CASE WHEN index <> index2 AND
                size(apoc.coll.intersection(acc, results[index2])) > 0
                THEN apoc.coll.union(acc, results[index2])
                ELSE acc
            END
    )) as combinedResult
    WITH distinct(combinedResult) as combinedResult
    // extra filtering
    WITH collect(combinedResult) as allCombinedResults
    UNWIND range(0, size(allCombinedResults)-1, 1) as combinedResultIndex
    WITH allCombinedResults[combinedResultIndex] as combinedResult, combinedResultIndex, allCombinedResults
    WHERE NOT any(x IN range(0,size(allCombinedResults)-1,1)
        WHERE x <> combinedResultIndex
        AND apoc.coll.containsAll(allCombinedResults[x], combinedResult)
    )
    RETURN combinedResult
    """, params={'distance': word_edit_distance})
potential_duplicate_candidates

[{'combinedResult': ['被保險人', '被保險人身故']},
 {'combinedResult': ['保單年度', '保險年齡', '第六保單年度屆滿後之每一保單週年日']},
 {'combinedResult': ['增額繳清保險金額', '購買增額繳清保險金額']},
 {'combinedResult': ['各年度之增值回饋分享金', '增值回饋分享金', '現金給付', '現金給付增值回饋分享金']}]

In [12]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
# from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

In [15]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """You are a data processing assistant. Your task is to identify duplicate entities in a list and decide which of them should be merged.
The entities might be slightly different in format or content, but essentially refer to the same thing. Use your analytical skills to determine duplicates.

Here are the rules for identifying duplicates:
1. Entities with minor typographical differences should be considered duplicates.
2. Entities with different formats but the same content should be considered duplicates.
3. Entities that refer to the same real-world object or concept, even if described differently, should be considered duplicates.
4. If it refers to different numbers, dates, or products, do not merge results
"""
user_template = """
Here is the list of entities to process:
{entities}

Please identify duplicates, merge them, and provide the merged list.
"""

from typing import List, Optional
from langchain_core.pydantic_v1 import BaseModel, Field

class DuplicateEntities(BaseModel):
    entities: List[str] = Field(
        description="Entities that represent the same object or real-world entity and should be merged"
    )


class Disambiguate(BaseModel):
    merge_entities: Optional[List[DuplicateEntities]] = Field(
        description="Lists of entities that represent the same object or real-world entity and should be merged"
    )


extraction_llm = llm.with_structured_output(
    Disambiguate
)

extraction_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt,
        ),
        (
            "human",
            user_template,
        ),
    ]
)

In [16]:
extraction_chain = extraction_prompt | extraction_llm

def entity_resolution(entities: List[str]) -> Optional[List[List[str]]]:
    return [
        el.entities
        for el in extraction_chain.invoke({"entities": entities}).merge_entities
    ]

In [17]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

merged_entities = []
with ThreadPoolExecutor(max_workers=10) as executor:
    # Submitting all tasks and creating a list of future objects
    futures = [
        executor.submit(entity_resolution, el['combinedResult'])
        for el in potential_duplicate_candidates
    ]

    for future in tqdm(
        as_completed(futures), total=len(futures), desc="Processing documents"
    ):
        to_merge = future.result()
        if to_merge:
            merged_entities.extend(to_merge)

Processing documents: 100%|██████████| 4/4 [00:01<00:00,  2.85it/s]


In [18]:
merged_entities

[['保單年度', '第六保單年度屆滿後之每一保單週年日'],
 ['增額繳清保險金額', '購買增額繳清保險金額'],
 ['各年度之增值回饋分享金', '增值回饋分享金'],
 ['現金給付', '現金給付增值回饋分享金'],
 ['被保險人', '被保險人身故']]

In [19]:
graph.query("""
UNWIND $data AS candidates
CALL {
  WITH candidates
  MATCH (e:__Entity__) WHERE e.id IN candidates
  RETURN collect(e) AS nodes
}
CALL apoc.refactor.mergeNodes(nodes, {properties: {
    description:'combine',
    `.*`: 'discard'
}})
YIELD node
RETURN count(*)
""", params={"data": merged_entities})

[{'count(*)': 5}]